In [1]:
from newsapi import NewsApiClient
import pandas as pd
import numpy as np1
from datetime import datetime, timedelta
import requests
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

load_dotenv()

False

In [2]:
newsapi = NewsApiClient(api_key="8c82dafbeadf4751b73b573fb7b6671e")
data = newsapi.get_sources()

# Classify sources by category
categories = {}
countries = {}

In [3]:
for source in data['sources']:
    category = source['category']
    country = source['country']
    
    if category not in categories:
        categories[category] = []
    categories[category].append(source)
    
    if country not in countries:
        countries[country] = []
    countries[country].append(source)

def create_dataframe(data_dict, outer_key_name, inner_key_name):
    records = []
    for outer_key, sources in data_dict.items():
        for source in sources:
            record = {
                outer_key_name: outer_key,
                inner_key_name: source['name'],
                'id': source['id'],
                'description': source['description'],
                'url': source['url'],
                'category': source['category'],
                'language': source['language'],
                'country': source['country']
            }
            records.append(record)
    return pd.DataFrame(records).set_index([outer_key_name, inner_key_name])

In [4]:
# Create DataFrame for categories
df_categories = create_dataframe(categories, 'Category', 'Source Name')
yesterday_date = (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d')
df_categories['yesterday_date'] = yesterday_date
df_categories.drop(labels='category', axis=1, inplace=True)

# Create DataFrame for countries
df_countries = create_dataframe(countries, 'Country', 'Source Name')

list_sources = ['abc-news', 'al-jazeera-english', 'bbc-news', 'google-news', 'google-news-in', 'politico', 'reuters', 'reddit-r-all', 'the-hindu', 'bloomberg', 
                'the-times-of-india', 'business-insider', 'vice-news', 'the-wall-street-journal', 'wired', 'national-geographic', 'bbc-sport', 'espn', 'techcrunch',
                'crypto-coins-news','buzzfeed','entertainment-weekly','medical-news-today','fortune','new-scientist','the-lad-bible', 'new-york-magazine', 'the-washington-times',
                'the-washington-post', 'cnn', 'ign', 'the-verge', 'fox-sports', 'talksport', 'the-sport-bible', 'hacker-news', 'recode', 'the-next-web', 'entertainment-weekly',
                'time', 'the-hindu', 'cbs-news', 'asutralian-financial-review', 'business-insider-uk', 'financial-post', 'fortune', 'info-money']

df_categories = df_categories[df_categories['id'].isin(list_sources)]

In [5]:
print(len(list_sources))

47


In [6]:
def fetch_json(source_id, date):
    url = (f'https://newsapi.org/v2/everything?'
           f'sources={source_id}&'
           f'from={date}&'
           'sortBy=popularity&'
           'apiKey=8c82dafbeadf4751b73b573fb7b6671e')
    
    response = requests.get(url)
    return response.json()


In [7]:
# def scrape_content(urls):
#     contents = []
#     for url in urls:
#         try:
#             response = requests.get(url)
#             soup = BeautifulSoup(response.content, 'html.parser')
#             article_content = soup.find_all('p')
#             content_text = ' '.join([paragraph.get_text() for paragraph in article_content])
#             contents.append(content_text)
#         except Exception as e:
#             contents.append(f"Error: {str(e)}")
#     return contents

def scrape_content(urls):
    contents = []
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')

            # Attempt to find the main content using common HTML structures
            article_content = soup.find('article')
            if not article_content:
                # Fallback to more general selectors if <article> is not found
                article_content = soup.find('div', class_='main-content')
            if not article_content:
                # Fallback to another common class name
                article_content = soup.find('div', class_='article-body')
            if not article_content:
                # As a last resort, get all paragraphs but this might include ads
                article_content = soup.find_all('p')
                content_text = ' '.join([paragraph.get_text() for paragraph in article_content])
            else:
                # Extract all paragraphs within the main content
                paragraphs = article_content.find_all('p')
                content_text = ' '.join([paragraph.get_text() for paragraph in paragraphs])

            contents.append(content_text)
        except Exception as e:
            contents.append(f"Error: {str(e)}")
    return contents

In [8]:
def json_to_df(data):
    try:
        articles = data['articles']
        if not articles:
            raise ValueError("No articles found for the given date.")
        
        df = pd.DataFrame(articles)
        df = df[['author', 'title', 'url', 'description', 'publishedAt']]
        df.columns = ['author', 'title', 'url', 'description', 'date']
        
        # Adding the content column by scraping the URLs
        df['content'] = scrape_content(df['url'])
        
        return df
    except KeyError as e:
        print(f"KeyError: {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame()

In [9]:
# Function to remove rows with empty content
def remove_empty_content(df):
    if 'content' in df.columns:
        # Removing rows with empty content
        df = df[df['content'].str.strip().astype(bool)]
        df = df.reset_index(drop=True)
    return df

In [10]:
n = 1
df_trial = df_categories.iloc[[n], :].copy()
df_trial

,,id,description,url,language,country,yesterday_date
Category,Source Name,,,,,,
general,Al Jazeera English,al-jazeera-english,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,en,us,2024-06-07


In [11]:
df_trial = df_categories.iloc[[n], :].copy()
df_trial['data'] = df_trial.apply(
    lambda row: json_to_df(fetch_json(row['id'], row['yesterday_date'])),
    axis=1
)

# Apply the function to clean up each DataFrame in the 'data' column
df_trial['data'] = df_trial['data'].apply(remove_empty_content)
# Remove rows where 'data' column has empty DataFrames
df_trial1 = df_trial[df_trial['data'].apply(lambda x: not x.empty)].copy()

In [12]:
df_trial1

,,id,description,url,language,country,yesterday_date,data
Category,Source Name,,,,,,,
general,Al Jazeera English,al-jazeera-english,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,en,us,2024-06-07,author ...


In [13]:
df_trial1['data'][0]

C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\3842372735.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0]


,author,title,url,description,date,content
0,Al Jazeera,Hospital overwhelmed with victims of Israeli a...,https://www.aljazeera.com/gallery/2024/6/8/hos...,At least 210 dead and more wounded in Israeli ...,2024-06-08T16:00:20Z,In Pictures Gaza’s Government Media Office say...
1,Gerald Nesmith Jr,The fallacy of the ‘wrong side of history’ nar...,https://www.aljazeera.com/opinions/2024/6/9/th...,Waiting on history to deliver karmic justice i...,2024-06-09T13:11:32Z,Waiting on history to deliver karmic justice f...
2,Al Jazeera,"Russia-Ukraine war: List of key events, day 836",https://www.aljazeera.com/news/2024/6/10/russi...,These are the main developments as the war ent...,2024-06-10T11:36:25Z,"As the war enters its 836th day, these are the..."
3,Al Jazeera,"Mexico heatwave killing monkeys, lions offered...",https://www.aljazeera.com/gallery/2024/6/11/me...,Dozens of howler monkeys were found dead in th...,2024-06-11T09:08:29Z,In Pictures Amid Mexico’s heatwave and drought...
4,Al Jazeera,Clashes erupt between police and protesters as...,https://www.aljazeera.com/gallery/2024/6/13/cl...,President Javier Milei seeks to pass a key bil...,2024-06-13T01:39:16Z,In Pictures Violent protests have erupted on t...
5,Al Jazeera,More than 80 dead in DR Congo after boat capsizes,https://www.aljazeera.com/news/2024/6/12/dozen...,"This is a breaking news story, more details to...",2024-06-12T14:54:58Z,President Felix Tshisekedi calls for investiga...
6,Al Jazeera,"Russia-Ukraine war: List of key events, day 837",https://www.aljazeera.com/news/2024/6/11/russi...,"As the war enters is 837th day, these are the ...",2024-06-11T01:52:51Z,"As the war enters is 837th day, these are the ..."
7,Al Jazeera,"Russia-Ukraine war: List of key events, day 838",https://www.aljazeera.com/news/2024/6/12/russi...,"As the war enters its 838th day, these are the...",2024-06-12T01:15:40Z,"As the war enters its 838th day, these are the..."
8,Al Jazeera,At least 35 dead in Kuwait building fire,https://www.aljazeera.com/news/2024/6/12/dozen...,Fire erupts in building in southern Mangaf dis...,2024-06-12T09:26:22Z,Fire erupts in building in southern Mangaf dis...
9,Al Jazeera,Photos: Nomadic Muslims throng forest shrine i...,https://www.aljazeera.com/gallery/2024/6/13/ph...,The 19th-century shrine to Mian Nizamuddin Kiy...,2024-06-13T11:10:12Z,In Pictures The road to the Baba Nagri forest ...


In [14]:
text = df_trial1['data'][0]['content'][5]

C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\3725325152.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = df_trial1['data'][0]['content'][5]


In [15]:
# Replace each period with a period followed by a newline character
formatted_text = text.replace('.', '.\n')

# Define the filename
filename = 'output.txt'

# Open the file in write mode and save the formatted text
with open(filename, 'w',encoding='utf-8') as file:
    file.write(formatted_text)

print(f"The text has been saved to {filename}")

The text has been saved to output.txt


In [16]:
df_trial

,,id,description,url,language,country,yesterday_date,data
Category,Source Name,,,,,,,
general,Al Jazeera English,al-jazeera-english,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,en,us,2024-06-07,author ...


In [19]:
df_categories

id  \
Category      Source Name                                        
general       ABC News                                abc-news   
              Al Jazeera English            al-jazeera-english   
              BBC News                                bbc-news   
              CBS News                                cbs-news   
              CNN                                          cnn   
              Google News                          google-news   
              Google News (India)               google-news-in   
              New York Magazine              new-york-magazine   
              Politico                                politico   
              Reddit /r/all                       reddit-r-all   
              Reuters                                  reuters   
              The Hindu                              the-hindu   
              The Times of India            the-times-of-india   
              The Washington Post          the-washington-post   
              The Washington Times        the-washington-times   
              Time                                        time   
              Vice News                              vice-news   
business      Bloomberg                              bloomberg   
              Business Insider                business-insider   
              Business Insider (UK)        business-insider-uk   
              Financial Post                    financial-post   
              Fortune                                  fortune   
              InfoMoney                             info-money   
              The Wall Street Journal  the-wall-street-journal   
technology    Crypto Coins News              crypto-coins-news   
              Hacker News                          hacker-news   
              Recode                                    recode   
              TechCrunch                            techcrunch   
              The Next Web                        the-next-web   
              The Verge                              the-verge   
              Wired                                      wired   
sports        BBC Sport                              bbc-sport   
              ESPN                                        espn   
              Fox Sports                            fox-sports   
              TalkSport                              talksport   
              The Sport Bible                  the-sport-bible   
entertainment Buzzfeed                                buzzfeed   
              Entertainment Weekly        entertainment-weekly   
              IGN                                          ign   
              The Lad Bible                      the-lad-bible   
health        Medical News Today            medical-news-today   
science       National Geographic          national-geographic   
              New Scientist                      new-scientist   

                                                                             description  \
Category      Source Name                                                                  
general       ABC News                 Your trusted source for breaking news, analysi...   
              Al Jazeera English       News, analysis from the Middle East and worldw...   
              BBC News                 Use BBC News for up-to-the-minute news, breaki...   
              CBS News                 CBS News: dedicated to providing the best in j...   
              CNN                      View the latest news and breaking news today f...   
              Google News              Comprehensive, up-to-date news coverage, aggre...   
              Google News (India)      Comprehensive, up-to-date India news coverage,...   
              New York Magazine        NYMAG and New York magazine cover the new, the...   
              Politico                 Political news about Congress, the White House...   
              Reddit /r/all            Reddit is an entertainment, social new

In [18]:
import os
import pandas as pd

# Assuming df_categories is already defined and contains the necessary data

# Define the directory where files will be saved
output_dir = 'output_files'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for n in range(43):
    try:
        df_trial = df_categories.iloc[[n], :].copy()
        df_trial['data'] = df_trial.apply(
            lambda row: json_to_df(fetch_json(row['id'], row['yesterday_date'])),
            axis=1
        )

        # Apply the function to clean up each DataFrame in the 'data' column
        df_trial['data'] = df_trial['data'].apply(remove_empty_content)
        
        # Remove rows where 'data' column has empty DataFrames
        df_trial1 = df_trial[df_trial['data'].apply(lambda x: not x.empty)].copy()
        
        # Check if df_trial1 is not empty and the 'content' column has sufficient elements
        if not df_trial1.empty and 'content' in df_trial1['data'].iloc[0] and len(df_trial1['data'].iloc[0]['content']) > 0:
            
            #make a csv of this df_trial1['data'][0]
            df_trial1['data'][0].to_csv(f'output_{n}.csv')

            text = df_trial1['data'].iloc[0]['content'][0]
            
            # Replace each period with a period followed by a newline character
            formatted_text = text.replace('.', '.\n')
            
            # Extract the URL
            url = df_trial1['data'].iloc[0]['url'][0]
            
            # Append the URL to the formatted text
            formatted_text += f"\n\nURL: {url}"
            
            # Define the filename
            filename = os.path.join(output_dir, f'output_{n}.txt')
            
            # Open the file in write mode and save the formatted text
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(formatted_text)
            
            print(f"The text for index {n} has been saved to {filename}")
        else:
            print(f"No sufficient content available for index {n}")
    except Exception as e:
        print(f"An error occurred for index {n}: {e}")


The text for index 0 has been saved to output_files\output_0.txt
The text for index 1 has been saved to output_files\output_1.txt
The text for index 2 has been saved to output_files\output_2.txt
The text for index 3 has been saved to output_files\output_3.txt
The text for index 4 has been saved to output_files\output_4.txt
The text for index 5 has been saved to output_files\output_5.txt
The text for index 6 has been saved to output_files\output_6.txt
The text for index 7 has been saved to output_files\output_7.txt
The text for index 8 has been saved to output_files\output_8.txt
ValueError: No articles found for the given date.
No sufficient content available for index 9
The text for index 10 has been saved to output_files\output_10.txt
ValueError: No articles found for the given date.
No sufficient content available for index 11
The text for index 12 has been saved to output_files\output_12.txt
The text for index 13 has been saved to output_files\output_13.txt
ValueError: No articles f

In [29]:
import os
import pandas as pd

# Assuming df_categories is already defined and contains the necessary data

# Define the directories where files will be saved
output_dir = 'output_files'
csv_output_dir = 'csv_output_files'

# Create the directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(csv_output_dir, exist_ok=True)

for n in range(43):
    try:
        df_trial = df_categories.iloc[[n], :].copy()
        df_trial['data'] = df_trial.apply(
            lambda row: json_to_df(fetch_json(row['id'], row['yesterday_date'])),
            axis=1
        )

        # Apply the function to clean up each DataFrame in the 'data' column
        df_trial['data'] = df_trial['data'].apply(remove_empty_content)
        
        # Remove rows where 'data' column has empty DataFrames
        df_trial1 = df_trial[df_trial['data'].apply(lambda x: not x.empty)].copy()
        
        # Check if df_trial1 is not empty and the 'content' column has sufficient elements
        if not df_trial1.empty and 'content' in df_trial1['data'].iloc[0] and len(df_trial1['data'].iloc[0]['content']) > 0:
            
            # Save a CSV of this df_trial1['data'][0]
            csv_filename = f'output_{n}.csv'
            df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))

            text = df_trial1['data'].iloc[0]['content'][0]
            
            # Replace each period with a period followed by a newline character
            formatted_text = text.replace('.', '.\n')
            
            # Extract the URL
            url = df_trial1['data'].iloc[0]['url'][0]
            
            # Append the URL to the formatted text
            formatted_text += f"\n\nURL: {url}"
            
            # Define the filename
            text_filename = os.path.join(output_dir, f'output_{n}.txt')
            
            # Open the file in write mode and save the formatted text
            with open(text_filename, 'w', encoding='utf-8') as file:
                file.write(formatted_text)
            
            print(f"The text for index {n} has been saved to {text_filename}")
        else:
            print(f"No sufficient content available for index {n}")
    except Exception as e:
        print(f"An error occurred for index {n}: {e}")


C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))


The text for index 0 has been saved to output_files\output_0.txt


C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))


The text for index 1 has been saved to output_files\output_1.txt


C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))


...

C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))


The text for index 40 has been saved to output_files\output_40.txt


C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))


The text for index 41 has been saved to output_files\output_41.txt
The text for index 42 has been saved to output_files\output_42.txt


C:\Users\ankur\AppData\Local\Temp\ipykernel_6816\2301266507.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_trial1['data'][0].to_csv(os.path.join(csv_output_dir, csv_filename))
